<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_color_space_skin_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 肌領域の検出（Skin detection）
----

氏名：

学生番号：

----
基本課題（必須）

    1. カラー画像にはどのような表現がありますか．各表現の利点・欠点を述べてください．

（ここに回答を書いてください）



    2. HSV空間にどのような条件を課すと肌領域を検出できますか．
       その条件はどのように見つけられますか．

（ここに回答を書いてください）



    3. 色だけを用いる肌領域の検出にはどのような問題点がありますか．また，どのような改善の工夫が考えられますか．

（[この文献](https://link.springer.com/article/10.1007/s10462-018-9664-9)を参考にして，回答を書いてください）



    4.その他，気づいたこと，調べたことがあれば書いてください．

（ここに回答を書いてください）



----


In [ ]:
#@title データセットをダウンロードします．
import zipfile
zipURL = "https://github.com/tsakailab/prml/raw/master/datasets/SkinDataset.zip"
!wget $zipURL --no-check-certificate --show-progress -q -O "/tmp/SkinDataset.zip"
with zipfile.ZipFile("/tmp/SkinDataset.zip", 'r') as f:
    f.extractall("./sample_data")


path_img = "./sample_data/SkinDataset/img"
path_lbl = "./sample_data/SkinDataset/lbl"
print("path_img = \"{}\"".format(path_img))
print("path_lbl = \"{}\"".format(path_lbl))

import os
imgfiles = [f for f in os.listdir(path_img) if os.path.isfile(os.path.join(path_img, f))]
print("imgfiles: list of filenames of " + str(len(imgfiles)) + " images in " + path_img)
lblfiles = [os.path.splitext(os.path.basename(f))[0]+".png" for f in imgfiles]
print("lblfiles: list of filenames of " + str(len(imgfiles)) + " skin masks in " + path_lbl)

import numpy as np
from PIL import Image
def read_images(filenames, path=path_img):
    imgs = []
    for f in filenames:
        img = Image.open(os.path.join(path, f))
        imgs.append(np.asarray(img))
    return imgs

### `img_id` で指定した画像と正解の肌領域を表示し，画像のNumpy配列 `img` から画素値 [R, G, B] の3次元ベクトルの集まり `rgb` を作ります．

In [ ]:
img_id = 1  # 0..99

import IPython.display as IPd
import os
im = IPd.Image(os.path.join(path_img, imgfiles[img_id]), width=300)
lb = IPd.Image(os.path.join(path_lbl, lblfiles[img_id]), width=300)
print(im.filename)
IPd.display(im)
print(lb.filename)
IPd.display(lb)

img = read_images([ imgfiles[img_id] ], path=path_img)[0]
lbl = read_images([ lblfiles[img_id] ], path=path_lbl)[0]
print(img.shape)

In [ ]:
rgb = np.reshape(img, (-1,3))
print("rgb.shape =", rgb.shape)

import pandas as pd
print(pd.DataFrame(data=rgb, columns=["Red", "Green", "Blue"]).head(10))

In [ ]:
#@title 色のヒストグラムを表示します．
import matplotlib.pyplot as plt

fig, axes = plt.subplots(figsize=(9,3), nrows=1, ncols=3, sharex=True, sharey=True)
axes[0].hist(rgb[:,0], bins = 256, color = 'red', alpha = 0.5)
axes[0].set_xlabel('Red')
axes[1].hist(rgb[:,1], bins = 256, color = 'green', alpha = 0.5)
axes[1].set_xlabel('Green')
axes[2].hist(rgb[:,2], bins = 256, color = 'blue', alpha = 0.5)
axes[2].set_xlabel('Blue')
plt.show()

### RGB色空間で観察します．
- `ns`: 色空間に散布する画素数（画素全部は多すぎるので）．

In [ ]:
ns = 3000

n = rgb.shape[0]
ns = min(n, ns)
print("Randomly chosen %d points out of %d points will be displayed." % (ns, n))

In [ ]:
#@title RGB色空間を表示します．マウスで視点を操作して観察できます．
import plotly.graph_objs  as go
import plotly.graph_objs.layout  as gol

ps = np.random.choice(n, ns, replace=False)
rgbs = rgb[ps] # * 1.5 # brighter

# https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scatter3d.html#plotly.graph_objects.Scatter3d
trace = go.Scatter3d(x=rgbs[:,0], y=rgbs[:,1], z=rgbs[:,2], mode='markers',
                     marker=dict(size=2, 
                                color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(rgbs[:,0], rgbs[:,1], rgbs[:,2])],
                                opacity=0.8))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0), scene=gol.Scene(xaxis=gol.scene.XAxis(title="Red"), yaxis=gol.scene.YAxis(title="Green"), zaxis=gol.scene.ZAxis(title="Blue")))
fig = go.Figure(data=[trace], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=1.5, y=1.5, z=0.8))
fig.update_layout(scene_camera=camera)
fig.show()

### ★ [HSV（Hue, Saturation, Value）](https://en.wikipedia.org/wiki/HSL_and_HSV)による表現に変換します．

In [ ]:
import matplotlib.colors as colors
hsv = np.reshape(colors.rgb_to_hsv(img/255.0), (-1,3))
print(hsv.shape)

print("hsv.shape =", hsv.shape)

import pandas as pd
print(pd.DataFrame(data=hsv, columns=["Hue", "Saturation", "Value"]).head(10))

In [ ]:
#@title HSVのヒストグラムを表示します．
import matplotlib.pyplot as plt

fig, axes = plt.subplots(figsize=(9,3), nrows=1, ncols=3, sharex=True, sharey=False)
axes[0].hist(hsv[:,0], bins = 256, color = 'gray', alpha = 0.5)
axes[0].set_xlabel('Hue')
axes[1].hist(hsv[:,1], bins = 256, color = 'gray', alpha = 0.5)
axes[1].set_xlabel('Saturation')
axes[2].hist(hsv[:,2], bins = 256, color = 'gray', alpha = 0.5)
axes[2].set_xlabel('Value')
plt.show()

In [ ]:
#@title HSV色空間を観察します．マウスで視点を操作して観察できます．
import plotly.graph_objs  as go
import plotly.graph_objs.layout  as gol

nmax = 10000
def ScatterHSV(hsv, rgbs=None):
    if hsv.shape[0] > nmax:
        hsvs = hsv[np.random.choice(hsv.shape[0], nmax, replace=False),:]
    else:
        hsvs = hsv
    if rgbs is None: rgbs = colors.hsv_to_rgb(hsvs) * 255.

    # https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scatter3d.html#plotly.graph_objects.Scatter3d
    trace = go.Scatter3d(x=hsvs[:,0], y=hsvs[:,1], z=hsvs[:,2], mode='markers',
                         marker=dict(size=2, 
                                color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(rgbs[:,0], rgbs[:,1], rgbs[:,2])],
                                opacity=0.8))
    layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0), 
                       scene=gol.Scene(
                           xaxis=gol.scene.XAxis(title="Hue", range=[0,1]), 
                           yaxis=gol.scene.YAxis(title="Saturation", range=[0,1]), 
                           zaxis=gol.scene.ZAxis(title="Value", range=[0,1])))
    fig = go.Figure(data=[trace], layout=layout)
    camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=1, y=-2, z=0.8))
    fig.update_layout(scene_camera=camera)
    fig.show()

hsvs = hsv[ps]
ScatterHSV(hsvs)

### HSV表現で肌の画素を選び出す方法を考えよう．
- NumPy配列 `hsv` （★のセルを参照）の行が画素に対応します．
- `hsv`の3つの列 Hue，Saturation，Value の値に条件を課してください．条件を満たす画素（`hsv`の行）の番号のNumPy配列を `skin_pos` とします．
    - 例えば，`hsv[:,2] > 0.2` と書くと，Value が 0.2を超える条件を表します．
- 条件を満たす画素の番号の配列は `skin_pos =`[`np.where(条件)[0]`](https://numpy.org/doc/stable/reference/generated/numpy.where.html)のように作成できます．
    - 行についての結果のみを使用するため末尾に`[0]`を付けています．
    - 複数の条件を課すときは，`skin_pos = np.where( `[`np.logical_and(条件1, ..., 条件n) `](https://numpy.org/doc/stable/reference/generated/numpy.logical_and.html)`)[0]`のように書きます．[`np.logical_or`](https://numpy.org/doc/stable/reference/generated/numpy.logical_or.html)などもあります．
    - [for文を書いたら負けかなと思ってる．](https://www.google.com/search?q=python+for+%E8%B2%A0%E3%81%91&rlz=1C1QABZ_jaJP862JP862&sxsrf=ALiCzsb_mbOOWtLLdk8-1feR5u76sZAMxw%3A1665660365256&ei=zfVHY4ioD-_s2roP8aqtiAY&ved=0ahUKEwjIwoyzjN36AhVvtlYBHXFVC2EQ4dUDCA4&uact=5&oq=python+for+%E8%B2%A0%E3%81%91&gs_lcp=Cgdnd3Mtd2l6EAMyBAgAEB46BwgAEB4QogQ6BQgAEKIEOgYIABAHEB5KBAhNGAFKBAhBGABKBAhGGABQAFidCmDADGgAcAF4AIABeYgBgAaSAQMxLjaYAQCgAQHAAQE&sclient=gws-wiz)

In [ ]:
#skin_pos = np.where( ここに条件を書く )[0]
skin_pos = np.where( np.logical_and(hsv[:,0] < 50/360., hsv[:,1] > 58/255., hsv[:,2] > 50/255.) )[0]
print(skin_pos.shape[0], "pixels are selected.")
ScatterHSV(hsv[skin_pos])

In [ ]:
#@title 検出結果を画像で表示します．

#est = np.ones_like(rgb) * 255
est = np.zeros_like(rgb)

est[skin_pos,:] = rgb[skin_pos,:]
est = np.reshape(est, img.shape)

gt = np.zeros_like(img[:,:,:3])
idx_lbl = lbl[:,:,:3] > 0
gt[idx_lbl] = img[:,:,:3][idx_lbl]

plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.imshow(img)
plt.gca().set_title("Original image")
plt.subplot(1,3,2)
plt.imshow(est)
plt.gca().set_title("Estimated skin pixels")
plt.subplot(1,3,3)
plt.imshow(gt)
plt.gca().set_title("Ground truth")
plt.show()

In [ ]:
#@title 混同行列（行：正解，列：予測）と評価値
from sklearn import metrics
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

y_true = idx_lbl[:,:,0].ravel()
y_pred = np.full_like(y_true, False)
y_pred[skin_pos] = True
cm = metrics.confusion_matrix(y_true, y_pred)
print(cm)

print(metrics.classification_report(y_true, y_pred, labels=[True], target_names=["skin pixels"]))

お疲れさまでした．